In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [83]:
df = pd.read_csv(r"https://raw.githubusercontent.com/JonathanBechtel/dat-02-22/main/ClassMaterial/Unit3/data/ks2.csv")

In [3]:
def split_data(df, split_frac=0.2, return_val=False, random_state=42):
    df = df.drop(['deadline', 'launched'], axis = 1)
    X  = df.drop('state', axis=1)
    y  = df['state']
    return train_test_split(X, y, test_size = split_frac, stratify = y, random_state = random_state)

In [4]:
def get_model_scores(mod, X_train, y_train, X_test, y_test, val_score = True, test_score = False):
    if val_score:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                          test_size = 0.2, 
                                                          stratify = y_train, 
                                                          random_state= 42)
 
    mod.fit(X_train, y_train)
    
    results = {}
    
    results['train_score'] = mod.score(X_train, y_train)
    if val_score:
        results['val_score'] = mod.score(X_val, y_val)
        
    if test_score:
        results['test_score'] = mod.score(X_test, y_test)
        
    return results

In [18]:
te   = ce.TargetEncoder(min_samples_leaf = 30)
ore  = ce.OrdinalEncoder()
mod = xgb.XGBClassifier()

pipe = make_pipeline(te, mod)

In [9]:
X_train, X_test, y_train, y_test = split_data(df)

In [20]:
scores = get_model_scores(pipe, X_train, y_train, X_test, y_test)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
scores

{'train_score': 0.7016027668817748, 'val_score': 0.6679769878359455}

In [21]:
feats = pd.DataFrame({
    'Features': X_train.columns,
    'Importance': pipe[-1].feature_importances_
})

feats

,Features,Importance
0,ID,0.034531
1,name,0.000000
2,category,0.523399
3,main_category,0.072589
4,currency,0.060460
5,country,0.103629
6,goal,0.205392


In [22]:
df

,ID,name,category,main_category,currency,deadline,launched,state,country,goal
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 00:00:00,2015-08-11 12:12:28,0,GB,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01 00:00:00,2017-09-02 04:43:57,0,US,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:00:00,2013-01-12 00:20:50,0,US,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 00:00:00,2012-03-17 03:24:11,0,US,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 00:00:00,2015-07-04 08:35:03,0,US,19500.00
...,...,...,...,...,...,...,...,...,...,...
370449,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17 00:00:00,2014-09-17 02:35:30,0,US,50000.00
370450,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19 00:00:00,2011-06-22 03:35:14,0,US,1500.00
370451,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16 00:00:00,2010-07-01 19:40:30,0,US,15000.00
370452,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13 00:00:00,2016-01-13 18:13:53,0,US,15000.00


In [84]:
df['launched'] = pd.to_datetime(df['launched'])
df['deadline'] = pd.to_datetime(df['deadline'])

In [85]:
df['launced_year']         = df['launched'].dt.year
df['launched_month']       = df['launched'].dt.month
df['launched_quarter']     = df['launched'].dt.quarter
df['launched_day']         = df['launched'].dt.day
df['launched_day_of_week'] = df['launched'].dt.weekday
df['duration']             = (df['deadline'] - df['launched']).dt.days
df['time']                 = (df['launched'] - df['launched'].min()).dt.days

In [86]:
X_train, X_test, y_train, y_test = split_data(df)
scores = get_model_scores(pipe, X_train, y_train, X_test, y_test)

In [33]:
scores

{'train_score': 0.7257792399510734, 'val_score': 0.7047897018878747}

In [34]:
feats = pd.DataFrame({
    'Features': X_train.columns,
    'Importance': pipe[-1].feature_importances_
})

feats.sort_values(by='Importance', ascending=False)

,Features,Importance
2,category,0.333805
6,goal,0.162387
13,duration,0.119909
5,country,0.078065
12,time,0.064590
4,currency,0.054774
3,main_category,0.048622
11,launched_day_of_week,0.041216
8,launched_month,0.027684
10,launched_day,0.024356


In [35]:
mod.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 12,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [87]:
train_idx = X_train.index

In [88]:
cat_avgs = df.iloc[train_idx].groupby('category')['goal'].mean().to_frame().rename({'goal': 'cat_avg'}, axis = 1)

In [89]:
df = df.merge(cat_avgs, left_on='category', right_index=True )

In [90]:
df['cat_avg'] = df['goal'] / df['cat_avg']

In [91]:
country_avgs = df.iloc[train_idx].groupby(['country', 'category'])['goal'].mean().to_frame().rename({'goal': 'country_avg'}, axis = 1)

In [92]:
df = df.merge(country_avgs, left_on=['country', 'category'], right_index=True )

In [94]:
df['country_avg'] = df['goal'] / df['country_avg']

In [95]:
X_train, X_test, y_train, y_test = split_data(df)
scores = get_model_scores(pipe, X_train, y_train, X_test, y_test)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:31:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [96]:
scores

{'train_score': 0.7261282469929586, 'val_score': 0.7062119243296151}

In [97]:
feats = pd.DataFrame({
    'Features': X_train.columns,
    'Importance': pipe[-1].feature_importances_
})

feats.sort_values(by='Importance', ascending=False)

,Features,Importance
2,category,0.354720
6,goal,0.139175
12,duration,0.110733
5,country,0.070327
14,cat_avg,0.055085
13,time,0.054418
11,launched_day_of_week,0.039108
3,main_category,0.037980
15,country_avg,0.033022
4,currency,0.028220


In [98]:
n_estimators = [100, 200, 300, 400, 500]
learning_rate = [0.1, 0.2, 0.3]
tree_depth = [3, 4]
subsample = [1, 0.8]
cv_scores = []

for estimator in n_estimators:
    for rate in learning_rate:
        for depth in tree_depth:
            for sample in subsample:
                print("Fitting new model")
                pipe[-1].set_params(n_estimators=estimator, learning_rate=rate, max_depth=depth, subsample=sample)
                scores = get_model_scores(pipe, X_train, y_train, X_test, y_test)
                cv_scores.append((scores['val_score'], scores['train_score'], estimator, rate, depth, sample))

Fitting new model


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:46:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting new model


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:46:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting new model


...

[20:49:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting new model


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting new model


C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [99]:
cv_scores

[(0.7006429620129268, 0.7032785294502314, 100, 0.1, 3, 1),
 (0.700963599237221, 0.7038058955308889, 100, 0.1, 3, 0.8),
 (0.7027524174359148, 0.7070586895163842, 100, 0.1, 4, 1),
 (0.7025667853586918, 0.7073666713074882, 100, 0.1, 4, 0.8),
 (0.7024992827851562, 0.7066072641513415, 100, 0.2, 3, 1),
 (0.7024824071417722, 0.7068139916549592, 100, 0.2, 3, 0.8),
 (0.7040518419764753, 0.7110033877997022, 100, 0.2, 4, 1),
 (0.7037818316823329, 0.7109021335122159, 100, 0.2, 4, 0.8),
 (0.7023305263513171, 0.7065988262940509, 200, 0.1, 3, 1),
 (0.7031236815903606, 0.7075818366683965, 200, 0.1, 3, 0.8),
 (0.7038155829691007, 0.7109738552991853, 200, 0.1, 4, 1),
 (0.7040687176198592, 0.7111552692309315, 200, 0.1, 4, 0.8),
 (0.7048281215721349, 0.7102017913571028, 200, 0.2, 3, 1),
 (0.7033261893109675, 0.7101764777852312, 200, 0.2, 3, 0.8),
 (0.7054018934471876, 0.7154459196631607, 200, 0.2, 4, 1),
 (0.7040349663330915, 0.7165133086104115, 200, 0.2, 4, 0.8)]

In [102]:
max(cv_scores)

(0.7054018934471876, 0.7154459196631607, 200, 0.2, 4, 1)

In [103]:
pipe[-1].set_params(n_estimators=200, learning_rate=0.2, max_depth=4, subsample=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [104]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7064033157376031